# Question 3

In [6]:
#-------------------------------------------------------------------------------
# importing the necessary libraries
import pandas as pd
import pickle
import os
import numpy as np


#reading in the NHANES data via the internet and adding column for cohort
year11 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/DEMO_G.XPT')
year11.insert(0,"Cohort",11)
year13 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/DEMO_H.XPT')
year13.insert(0,"Cohort",13)
year15 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.XPT')
year15.insert(0,"Cohort",15)
year17 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.XPT')
year17.insert(0,"Cohort",17)
sets = [year11, year13, year15, year17]

#creating a dataframe and filterng out for just the data we want
data = pd.concat(sets)
labels = ['Cohort','SEQN','RIDAGEYR','RIDRETH3','DMDEDUC2','DMDMARTL',
         'RIDSTATR','SDMVPSU','SDMVSTRA','WTMEC2YR','WTINT2YR','RIAGENDR']
data = data[labels]

#renaming columns
new_cols=[
    'cohort','unique_id',
    'age','race_ethnicity',
    'education','marital_status',
    'status','psuedo_PSU',
    'psuedo_stratum','exam_weight',
    'interview_weight', 'gender'
]

new_names_map = {data.columns[i]:new_cols[i] for i in range(len(new_cols))}

data.rename(new_names_map, axis=1, inplace=True)


    
    

# for the categorical data, specifying what each category represents
data['race_ethnicity'] = pd.Categorical(data['race_ethnicity'].replace(
        {
            1: 'Mexican American',
            2: 'Other Hispanic',
            3: 'Non-Hispanic White',
            4: 'Non-Hispanic Black',
            6: 'Non-Hispanic Asian',
            7: 'Other Race - Including Multi-Racial',
            np.nan: 'Missing'
        }))

data['education'] = pd.Categorical(data['education'].replace(
        {
            1: 'Less than 9th grade',
            2: '9-11th grade (Includes 12th grade with no diploma)',
            3: 'High school graduate/GED or equivalent',
            4: 'Some college or AA degree',
            5: 'College graduate or above',
            7: 'Refused',
            9: "Don't know",
            np.nan : 'Missing'
            
        }))


data['marital_status'] = pd.Categorical(data['marital_status'].replace(
        {
            1: 'Married',
            2: 'Widowed',
            3: 'Divorced',
            4: 'Separated',
            5: 'Never Married',
            6: 'Living with partner',
            77: 'Refused',
            99: "Don't know",
            np.nan : 'Missing'
            
        }))

data['status'] = pd.Categorical(data['status'].replace(
        {
            1: 'Interviewed only',
            2: 'Both interviewed and MEC examined',
            np.nan : 'Missing'
            
        }))

data['gender'] = pd.Categorical(data['gender'].replace(
        {
            1: 'Male',
            2: 'Female',
            np.nan : 'Missing'
            
        }))


#using pickle to dump the dataframe we just created into the working directory.
filepath = os.path.abspath('')
pickle.dump(data,open(os.path.join(filepath,'demo_data'),'wb'))

In [7]:
#-------------------------------------------------------------------------------
import pandas as pd
def dental(df = False):
    """
Creates and filters/edits the dataframe for NHANES dental data from 2011-2017

Parameters
----------
df: False, the default argument, will just write the pickled file to the working
directory. True will do the same thing but also return a dataframe.

 Returns
 -------
 Pickled df to working directory
 Cleaned up dataframe with labeled categoeries
 """

#scrape the data from the web for each year we need and insert cohort year    
    year11 = pd.read_sas(
        'https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/OHXDEN_G.XPT'
    )
    year11.insert(0,"cohort",11)
    year13 = pd.read_sas(
        'https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/OHXDEN_H.XPT'
    )
    year13.insert(0,"cohort",13)
    year15 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/OHXDEN_I.XPT'
                        )
    year15.insert(0,"cohort",15)
    year17 = pd.read_sas(
        'https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/OHXDEN_J.XPT'
    )
    year17.insert(0,"cohort",17)

#combine the scrapes dfs
    sets = [year11, year13, year15, year17]

    data = pd.concat(sets)



    labels = ['cohort','SEQN','OHDDESTS']
# create lists of label names for each of the 32 teeth and category. 
    tooth_count = []
    coronal_carries = []
    for i in range(32):
        if i<9:
            coronal_carries.append('OHX0{0}CTC'.format(i+1))
            tooth_count.append('OHX0{0}TC'.format(i+1))
        else:
            coronal_carries.append('OHX{0}CTC'.format(i+1))
            tooth_count.append('OHX{0}TC'.format(i+1))
            
#website missing some variables, dropping four hypothetical categories
    removelist = ['OHX01CTC','OHX16CTC', 'OHX17CTC', 'OHX32CTC']  



    for i in removelist:
        coronal_carries.remove(i)


    labels += tooth_count
    labels += coronal_carries


    data = data[labels]

# create categorical labels for each categorical variable

    data['OHDDESTS'] = pd.Categorical(data['OHDDESTS'].replace(
        {
            1: 'complete', 
            2: 'partial',
            3: 'not done'
        }))


    for i in tooth_count:
        data[i] = pd.Categorical(data[i].replace(
        {
            1: 'Primary tooth (deciduous) present',
            2: 'Permanent tooth present',
            3: 'Dental implant',
            4: 'Tooth not present',
            5: 'Permanent dental root fragment present',
            9: 'Could not assess'
        }))

    for j in coronal_carries:
        data[j] = pd.Categorical(data[j].replace(
        {
            b'A': 'Primary tooth with a restored surface condition',
            b'D': 'Sound primary tooth',
            b'E': 'Missing due to dental disease',
            b'F': 'Permanent tooth with a restored surface condition',
            b'J': 'Permanent root tip is present but no restorative replacement is present',
            b'K': 'Primary tooth with a dental carious surface condition',
            b'M': 'Missing due to other causes',
            b'P': 'Missing due to dental disease but replaced by a removable restoration',
            b'Q': 'Missing due to other causes but replaced by a removable restoration',
            b'R': 'Missing due to dental disease but replaced by a fixed restoration',
            b'S': 'Sound permanent tooth',
            b'T': 'Permanent root tip is present but a restorative replacement is present',
            b'U': 'Unerupted',
            b'X': 'Missing due to other causes but replaced by a fixed restoration',
            b'Y': 'Tooth present, condition cannot be assessed',
            b'Z': 'Permanent tooth with a dental carious surface condition'
        }))
        
# dumping the df in the working directory for future use        
    filepath = os.path.abspath('')
    pickle.dump(data,open(os.path.join(filepath,'dental_data'),'wb'))

#if the df is True, return the df we just created
    if df == True:
        return data

In [8]:
#-------------------------------------------------------------------------------
#Number of cases for the demo data
#this is for part c.

print('Demo Data has ' +str(data.shape[0]) + ' cases.')

#Number of cases for the dental data
dental1 = dental(True)
print('Dental Data has ' + str(dental1.shape[0]) +' cases.')

Demo Data has 39156 cases.
Dental Data has 35909 cases.
